#### Load and Prepare Data

In [2]:
import torch
import torchvision
from torchvision import datasets, transforms
import tkinter as tk
from PIL import Image, ImageGrab, ImageTk, ImageDraw

print(torch.__version__)

2.0.1+cpu


In [3]:
# Setup the data transformation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale = (0.8, 1.0)),
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

data_path = "images"

datasets = datasets.ImageFolder(root = data_path, transform = transform)

In [4]:
# Display class names and index labels for debug
print("Class names: ", datasets.classes)
print("Class index mapping:", datasets.class_to_idx)

Class names:  ['Poop', 'Smile']
Class index mapping: {'Poop': 0, 'Smile': 1}


In [5]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(datasets))
valid_size = len(datasets) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(datasets, [train_size, valid_size])

# Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=4, shuffle = False)


#### Define the Model

In [6]:
import torch.nn as nn
import torch.nn.functional as F

In [27]:
# Set up the neural net
class SimpleCNN(nn.Module):
    def __init__ (self, num_classes):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding = 1)
        self.fc1 = nn.Linear(32*32*32, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # first conv, ReLU, pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # second conv, ReLU, pooling
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten the tensor
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)
        return x
    
num_classes = len(datasets.classes)
model = SimpleCNN(num_classes)

#### Specify Loss Function and Optimizer

In [28]:
lossf = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


#### Train the Model

In [30]:
epochs = 100

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossf(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    if(epoch % 10 == 0):
        print(f"Epoch {epoch +1}/{epochs}, Loss: {train_loss: .4f}")

torch.save(model, "models/model1.pth")



Epoch 1/100, Loss:  0.5360
Epoch 11/100, Loss:  0.5338
Epoch 21/100, Loss:  0.5524
Epoch 31/100, Loss:  0.5503
Epoch 41/100, Loss:  0.5424
Epoch 51/100, Loss:  0.5434


#### Evaluate the Model

In [22]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in valid_loader:
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 63.64%
Test Accuracy: 77.52%


#### Live Prediction

In [23]:
# Assuming the SimpleCNN class definition is already present
model = torch.load('models/model1.pth')
model.eval()

SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=32768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [24]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Assuming 128x128 is the input size your model expects
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Standard normalization values, adjust if yours are different
])

In [25]:
class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Prediction Board")

        self.canvas = tk.Canvas(root, bg="black", width=500, height=500)
        self.canvas.pack(pady=20)
        
        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset_last_coordinates)

        # Button to make predictions
        self.predict_button = tk.Button(root, text="Predict", command=self.predict)
        self.predict_button.pack(pady=20)

        self.last_x, self.last_y = None, None
        self.filename = 0

    def paint(self, event):
        x, y = event.x, event.y
        if self.last_x and self.last_y:
            self.canvas.create_line((self.last_x, self.last_y, x, y), width=10, fill='white', capstyle=tk.ROUND, smooth=tk.TRUE)
        self.last_x, self.last_y = x, y

    def reset_last_coordinates(self, event):  
        self.last_x, self.last_y = None, None

    def clear_screen(self):
        self.canvas.delete("all")

    def predict(self):
        # Capture the canvas content and convert it to a PIL Image
        x = self.root.winfo_rootx() + self.canvas.winfo_x()
        y = self.root.winfo_rooty() + self.canvas.winfo_y()
        x1 = x + self.canvas.winfo_width()
        y1 = y + self.canvas.winfo_height()

        img = ImageGrab.grab(bbox=(x, y, x1, y1)).convert('RGB')
        
        # Transform the image and make a prediction
        img_tensor = transform(img).unsqueeze(0)
        
        with torch.no_grad():
            output = model(img_tensor)
            prediction = output.argmax(dim=1).item()
        
        # Display the prediction (this assumes the dataset classes are available)
        print(f"Predicted Class: {datasets.classes[prediction]}")
        
        probability = F.softmax(output, dim = 1)
        predicted_class = torch.argmax(probability, dim = 1)
        confidence = probability[0][predicted_class].item()
        print(f"Confidence: {confidence}")


In [26]:
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawingApp(root)
    root.mainloop()

Predicted Class: Smile
Confidence: 0.5216835737228394
